# Домашнее задание. Нейросетевая классификация текстов

В этом домашнем задании вам предстоит самостоятельно решить задачу классификации текстов. Вы будете классифицировать тексты писателей по авторству. Всего 8 разных авторов.

Скачаем датасет:

In [ ]:
!wget -O train_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors_new.csv

--2025-04-16 11:08:41--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors_new.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5163531 (4.9M) [text/plain]
Saving to: ‘train_authors.csv’

train_authors.csv   100%[===================>]   4.92M  28.2MB/s    in 0.2s    

2025-04-16 11:08:42 (28.2 MB/s) - ‘train_authors.csv’ saved [5163531/5163531]



In [ ]:
!wget -O test_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors_new.csv

--2025-04-16 11:08:44--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors_new.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 961703 (939K) [text/plain]
Saving to: ‘test_authors.csv’

test_authors.csv    100%[===================>] 939.16K  --.-KB/s    in 0.1s    

2025-04-16 11:08:44 (7.34 MB/s) - ‘test_authors.csv’ saved [961703/961703]



Импортируем необходимые библиотеки:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
from tqdm import tqdm_notebook
import torch.nn as nn                                                             # тут все блоки нейронных сетей, слои
import torch.nn.functional as F

from collections import Counter
from typing import List
import string

import seaborn
seaborn.set(palette='summer')

# скачиваем нужный пакет данных для работы библиотеки nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Загрузим даатсет и посмотрим на данные:

In [ ]:
# Загрузим датасет
import pandas as pd
train_data = pd.read_csv('train_authors.csv')
train_data.head()

,text,author
0,"Студент, который все это нам рассказал, прилет...",Bulychev
1,"-Что не укладывается в голове,- произнес отец ...",Pratchett
2,"-Ш-ш-ш,- сказал я.- Спи.\n-Не могу,- ответила ...",King
3,-В Севастополь? Новое задание имеет отношение ...,Akunin
4,"-Ты прав,- сказал Сева.- Даже если это космиче...",Bulychev


Здесь поле 'text' — это текст, а 'author' — автор текста. 'author' — это целевая переменная, которую мы будем предсказывать. Обратите внимания, что в тестовых данных столбца 'author' нет, вам нужно будет предсказать его значения и отправить в качестве ответа на Яндекс.Контест.

In [ ]:
test_data = pd.read_csv('test_authors.csv')
test_data.head()

,text
0,-Не выглядывал еще. Дрыхнет… Почему да почему?...
1,"Идти ему было немного; он даже знал, сколько ш..."
2,Придется вернуться в самое изголовье и начать ...
3,"-С ума сойти,- сказал отец. Он обернулся, увид..."
4,"Во-вторых, положение Петра после захвата власт..."


Заведем словарь соответствия имени автора его номеру:

In [ ]:
writers = ['Akunin', 'Bulychev', 'Chehov', 'Dostoevsky', 'Gogol', 'King',
       'Pratchett', 'Remark']
writers_to_label = {writer: i for i, writer in enumerate(writers)}
label_to_writers = {i: writer for i, writer in enumerate(writers)}

Создадим словарь dataset, который будет устроен так же, как переменная dataset в ноутбуке занятия. В тестовой части (dataset['test']) зададим для удобства всем текстам author=0.

In [ ]:
dataset = {}

dataset['train'] = [{'text':text, 'author':writers_to_label[label]} \
              for text, label in zip(np.array(train_data['text']), np.array(train_data['author']))]
dataset['test'] = [{'text':text, 'author': 0} \
              for text in np.array(test_data['text'])]

Ваша задача — обучить RNN-модель на тренировочных данных и получить максимальное возможное accuracy на тестовой части данных. За основу можно взять код с занятия. **Обратите внимание, что здесь у нас задача классификации на 8 классов, а не на 2, как было на занятии.**

Чтобы улучшить качество базовой модели, можно попробовать различные идеи экспериментов. Вот несколько идей:
* **Модель RNN**. Попробуйте LSTM и GRU. Мы советуем обратить внимание на [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html), так как интерфейс этого класса ничем не отличается от обычной Vanilla RNN, которую мы использовали на семинаре.
* **Увеличение количества рекуррентных слоев модели**. Это можно сделать с помощью параметра `num_layers` в классе `nn.RNN`. В такой модели выходы первой RNN передаются в качестве входов второй RNN и так далее.
* **Изменение вида агрреграции выхода RNN**. В семинаре мы пробовали аггрегировать выходы RNN с помощью max, mean и получения последнего выхода. Можно попробовать другие варианты. Например, конкатенировать результат агрегации и эмбеддинг с последнего токена.
* **Подбор гиперпараметров и обучение до сходимости**. Можно, например, увеличить количество эпох обучения нейросети, а также попробовать различные гиперпараметры: размер словаря, `dropout_rate`, `hidden_dim`.

In [ ]:
dataset['train'][0]['text'], dataset['train'][0]['author']

('Студент, который все это нам рассказал, прилетел на Стеговию, чтобы изучить обстановку и понять, как помочь тамошним животным. Но животные встретили его как злейшего врага. Оказывается, недавно на той планете побывали браконьеры, которые охотились на динозавров из-за их красивых шкур и рогов, а также собирали яйца ящеров. Теперь животные планеты Стеговия боятся людей и ненавидят их. Так что наш друг студент, который не знал о браконьерах, чуть не погиб, когда на него напали динозавры. Он еле унес ноги с планеты. Он обратился в Службу охраны галактической природы, и они поставили экспедицию на планету Стеговия в план второго квартала будущего года. У службы охраны много планет, которые надо защищать и охранять.\nТеперь ты понимаешь, Крыс, что мы с Пашкой Гераскиным решили сделать. Мы немедленно летим на планету Стеговия и постараемся хоть кого-то спасти. Но для того, чтобы это сделать, нам надо принять вид каких-то местных существ. Ведь мы не можем прилететь туда в человеческом облике

In [ ]:
import shutil
shutil.rmtree('/root/nltk_data/tokenizers/punkt', ignore_errors=True)
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import string
from collections import Counter
from tqdm.auto import tqdm

# Функция предобработки текста: нижний регистр, удаление пунктуации, токенизация
def process_and_tokenize_text(text):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    return text.split()  # простая токенизация по пробелам

# Инициализация списков и счётчика слов
train_data = []
test_data = []
words = Counter()

# Обработка обучающего набора
for example in tqdm(dataset['train'], desc="Processing train data"):
    text = example['text']
    label = example['author']
    text_processed = process_and_tokenize_text(text)
    train_data.append((text_processed, label))
    words.update(text_processed)

# Обработка тестового набора
for example in tqdm(dataset['test'], desc="Processing test data"):
    text = example['text']
    label = example['author']
    text_processed = process_and_tokenize_text(text)
    test_data.append((text_processed, label))


Processing train data:   0%|          | 0/1734 [00:00<?, ?it/s]

Processing test data:   0%|          | 0/330 [00:00<?, ?it/s]

In [ ]:
def collate_fn_with_padding(input_batch, max_len=256):
    # input_batch — батч пар (токенизированный текст, label)
    texts = [x[0] for x in input_batch]
    labels = [x[1] for x in input_batch]
    # для каждого элемента батча получаем длину токенизированного текста в токенах
    seq_lens = [len(x) for x in texts]
    # определяем максимальную длину элементов в текущем батче
    max_seq_len = min(max(seq_lens), max_len)

    # проходимся по элементам батча и заменяем токены на их индексы из словаря
    # также последовательности короче max_seq_len дополняем токенами <pad>
    processed_texts = []
    for text, label in zip(texts, labels):
        text = text[:max_seq_len]
        text = [word2ind[x] if x in vocab else word2ind['<unk>'] for x in text]
        for _ in range(max_seq_len - len(text)):
            text.append(word2ind['<pad>'])

        processed_texts.append(text)

    # переводим элементы батча в форматы тензоров
    processed_texts = torch.LongTensor(processed_texts).to(device)
    labels = torch.LongTensor(labels).to(device)

    # собираем батч
    processed_batch = {
        'input_ids': processed_texts,
        'label': labels
    }

    return processed_batch

In [ ]:
# поделим тестовые данные на val и test
np.random.seed(42)
val_indices = np.random.choice(np.arange(len(test_data)), 10000)
test_indices = [x for x in range(len(test_data)) if x not in val_indices]
val_data = [test_data[i] for i in val_indices]

# создадим словарь только из тех слов, которые встречаются чаще одного раза
vocab = [word for word, count in words.items() if count >= 2]

# добавим специальные токены
vocab = ['<pad>', '<unk>'] + vocab

# создадим отображения: слово -> индекс и наоборот
word2ind = {word: idx for idx, word in enumerate(vocab)}
ind2word = {idx: word for word, idx in word2ind.items()}
# заводим даталоадеры
batch_size = 32
train_dataloader = DataLoader(
    train_data, shuffle=True, collate_fn=collate_fn_with_padding, batch_size=batch_size)

val_dataloader = DataLoader(
    val_data, shuffle=False, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_data, shuffle=False, collate_fn=collate_fn_with_padding, batch_size=batch_size)

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(
        self, hidden_dim, vocab_size, num_classes,
        aggregation_type: str = 'last'
        ):
        super().__init__()
        # слой эмбеддинга
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        # рекуррентный слой
        # Параметры - размер эмбеддингов токенов, размер вектора скрытого состояния, формат представления данных в батче
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        # два полносвязных слоя
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

        # слой дропаут
        self.dropout = nn.Dropout(p=0.1)

        # каким образом выходы RNN-слоя для разных моментов времени будут
        # агрегироваться перед подачей дальше на вход полносвязному слою
        self.aggregation_type = aggregation_type

    def forward(self, input_batch) -> torch.Tensor:
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        output, _ = self.rnn(embeddings)          # [batch_size, seq_len, hidden_dim]

        if self.aggregation_type == 'max':
            output = output.max(dim=1)[0]         #[batch_size, hidden_dim]
        elif self.aggregation_type == 'mean':
            output = output.mean(dim=1)           #[batch_size, hidden_dim]
        elif self.aggregation_type == 'last':
            output = output[:, -1, :]
        else:
            raise ValueError("Invalid aggregation_type")

        output = F.tanh(output)
        output = F.tanh(self.dropout(self.fc1(output)))   # [batch_size, hidden_dim]
        output = self.fc2(output)                         # [batch_size, num_classes]

        return output

In [ ]:
def evaluate(model, dataloader):
    """
    Calculate accuracy on data from dataloader.
    """

    predictions = []
    target = []
    with torch.no_grad():
        for batch in tqdm_notebook(dataloader,
                                   desc=f'Evaluating'):
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))
            target.append(batch['label'])

    predictions = torch.cat(predictions)
    target = torch.cat(target)
    accuracy = (predictions == target).float().mean().item()

    return accuracy

In [ ]:
def train(model, optimizer, criterion, num_epoch=5, eval_steps=100):

    losses = []
    accs_train = []
    accs_val = []

    for epoch in range(num_epoch):
        epoch_losses = []
        model.train()
        for i, batch in enumerate(tqdm_notebook(train_dataloader,
                                                desc=f'Training epoch {epoch}:')):
            optimizer.zero_grad()
            logits = model(batch['input_ids'])
            loss = criterion(logits, batch['label'])
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
            if i % eval_steps == 0:
                model.eval()
                accs_train.append(evaluate(model, train_dataloader))
                accs_val.append(evaluate(model, val_dataloader))
                model.train()

        losses.append(sum(epoch_losses) / len(epoch_losses))

    return losses, accs_train, accs_val

In [ ]:
num_epoch = 1
eval_steps = len(train_dataloader) // 2


losses_type = {}
accs_train_type = {}
accs_val_type = {}

for aggregation_type in ['max', 'mean', 'last']:
    print(f"Starting training for {aggregation_type}")
    losses = []
    acc = []

    model = SimpleRNN(hidden_dim=256,
                      vocab_size=len(vocab),
                      num_classes=8,
                      aggregation_type=aggregation_type
                      ).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    optimizer = torch.optim.Adam(model.parameters())

    losses, accs_train, accs_val = train(model,
                                         optimizer,
                                         criterion,
                                         num_epoch=1,
                                         eval_steps=len(train_dataloader) // 2)

    losses_type[aggregation_type] = losses
    accs_train_type[aggregation_type] = accs_train
    accs_val_type[aggregation_type] = accs_val

Starting training for max


<ipython-input-72-48d3f6fda81d>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(train_dataloader,


Training epoch 0::   0%|          | 0/55 [00:00<?, ?it/s]

<ipython-input-71-fa459c92f69e>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(dataloader,


Evaluating:   0%|          | 0/55 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/55 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/55 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/313 [00:00<?, ?it/s]

## Сдача задания

Ниже приведена функция, которую вам необходимо запустить для обученной модели, чтобы получить предсказания на тестовой выборке. Здесь model — ваша обученная модель, dataloader — test_dataloader, построенный на основе тестовой части данных (dataset['test']):

In [ ]:
def get_predictions(model, dataloader):
    """
    Calculate accuracy on data from dataloader.
    """

    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm_notebook(dataloader,
                                   desc=f'Evaluating'):
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))

    predictions = torch.cat(predictions).data.cpu().numpy()

    return predictions

In [ ]:
predictions = get_predictions(model, test_dataloader)
predictions = [label_to_writers[x] for x in predictions]
predictions

<ipython-input-77-5a7bf3723a28>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(dataloader,


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

['Bulychev',
 'Dostoevsky',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'King',
 'King',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'King',
 'Dostoevsky',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'King',
 'Bulychev',
 'King',
 'Dostoevsky',
 'King',
 'Bulychev',
 'King',
 'King',
 'Bulychev',
 'King',
 'Bulychev',
 'Bulychev',
 'King',
 'King',
 'Bulychev',
 'Bulychev',
 'Bulychev

In [ ]:
print(f"test_data size: {len(test_data)}")


test_data size: 330


In [ ]:
np.save('submission_hw07.npy', predictions, allow_pickle=True)
print('Ответ сохранен в файл `submission_hw07.npy`')

Ответ сохранен в файл `submission_hw07.npy`


In [ ]:
import nltk
import shutil

# Удалим старую версию punkt
shutil.rmtree('/root/nltk_data/tokenizers/punkt', ignore_errors=True)

# Заново скачаем
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

# Настройка устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загрузка и подготовка данных
writers = ['Akunin', 'Bulychev', 'Chehov', 'Dostoevsky', 'Gogol', 'King', 'Pratchett', 'Remark']
writers_to_label = {writer: i for i, writer in enumerate(writers)}
label_to_writers = {i: writer for writer, i in writers_to_label.items()}

train_df = pd.read_csv("train_authors.csv")
test_df = pd.read_csv("test_authors.csv")

# Разделение train/val
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42)

# Загрузка токенизатора RuBERT
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

# Кастомный Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, max_len=256):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }
        if self.labels is not None:
            item['label'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Подготовка датасетов
train_dataset = TextDataset(train_data['text'].tolist(),
                            [writers_to_label[y] for y in train_data['author']])
val_dataset = TextDataset(val_data['text'].tolist(),
                          [writers_to_label[y] for y in val_data['author']])
test_dataset = TextDataset(test_df['text'].tolist())

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Модель на основе BERT
class BertClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = output.pooler_output
        x = self.dropout(pooled)
        return self.fc(x)

model = BertClassifier(num_classes=8).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Функции обучения и оценки
def evaluate(model, dataloader):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            logits = model(input_ids, attention_mask)
            preds.extend(logits.argmax(dim=1).cpu().numpy())
            targets.extend(labels.cpu().numpy())
    return classification_report(targets, preds, target_names=writers, digits=4)

def train(model, train_loader, val_loader, optimizer, criterion, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch} | Loss: {total_loss:.4f}")
        print(evaluate(model, val_loader))

# Обучение модели
train(model, train_loader, val_loader, optimizer, criterion, epochs=3)

# Предсказания на тесте
def predict(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            logits = model(input_ids, attention_mask)
            predictions.extend(logits.argmax(dim=1).cpu().numpy())
    return predictions

preds = predict(model, test_loader)
pred_labels = [label_to_writers[i] for i in preds]
np.save("submission_hw07_rubert.npy", pred_labels, allow_pickle=True)
print("✅ Предсказания сохранены в submission_hw07_rubert.npy")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2:   0%|          | 0/98 [00:00<?, ?it/s]

Epoch 2 | Loss: 34.6100


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      Akunin     0.8696    0.9524    0.9091        42
    Bulychev     0.8333    0.9615    0.8929        26
      Chehov     0.6800    0.9444    0.7907        18
  Dostoevsky     0.0000    0.0000    0.0000         1
       Gogol     1.0000    0.0909    0.1667        11
        King     0.9474    0.8780    0.9114        41
   Pratchett     0.8800    0.9167    0.8980        24
      Remark     1.0000    0.8182    0.9000        11

    accuracy                         0.8621       174
   macro avg     0.7763    0.6953    0.6836       174
weighted avg     0.8758    0.8621    0.8407       174

✅ Предсказания сохранены в submission_hw07_rubert.npy


In [ ]:
pred_labels

['Bulychev',
 'Dostoevsky',
 'Akunin',
 'Bulychev',
 'Akunin',
 'Pratchett',
 'Pratchett',
 'Akunin',
 'Bulychev',
 'Pratchett',
 'Akunin',
 'Akunin',
 'Chehov',
 'Chehov',
 'King',
 'Pratchett',
 'Remark',
 'Bulychev',
 'Akunin',
 'Pratchett',
 'Akunin',
 'Bulychev',
 'Akunin',
 'Chehov',
 'Akunin',
 'Bulychev',
 'Akunin',
 'King',
 'Chehov',
 'Bulychev',
 'Akunin',
 'Pratchett',
 'Akunin',
 'King',
 'Pratchett',
 'Bulychev',
 'Akunin',
 'Pratchett',
 'Chehov',
 'Remark',
 'King',
 'King',
 'King',
 'King',
 'Remark',
 'King',
 'Akunin',
 'Pratchett',
 'Akunin',
 'Akunin',
 'Pratchett',
 'King',
 'Pratchett',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Bulychev',
 'Akunin',
 'Akunin',
 'Akunin',
 'Pratchett',
 'Akunin',
 'Akunin',
 'Bulychev',
 'King',
 'Bulychev',
 'Pratchett',
 'Pratchett',
 'King',
 'Remark',
 'Akunin',
 'Chehov',
 'Chehov',
 'King',
 'King',
 'Akunin',
 'Pratchett',
 'Akunin',
 'Pratchett',
 'Chehov',
 'Bulychev',
 'Akunin',
 'Chehov',
 'Akunin',
 'Chehov',
 'Bulyche